In [ ]:
''' The outputs generated by the lab.js contain a number of columns, which may not be required for our
analysis. This basic notebook extracts relevant fields, removes identifying information, 
and converts all the delay information to a common time scale. 
The result is then written into separate .csvs, which can be used for model building and further analysis.
Author: Aishwarya Seth
Email: aseth5@jhu.edu
Date of Creation: 7th April, 2022'''

In [1]:
# Import all relevant packages for this step
import pandas as pd 
import numpy as np
import glob as glob
import seaborn as sns 

/Users/aishwaryaseth/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [49]:
# Collect all files containing experimental data
files = glob.glob("./../Experimental_Data/*.csv")

In [50]:
# The file_count is used as a unique identifier for each participant
file_count = 1

In [92]:
# For each file, 
for file in files: 
    # Read in all rows in the output dataframe
    df = pd.read_csv(file)
    
    # Select rows with responses for the Form, using 'sender' value 'Form' & store the row in df_info
    # Select rows with choice information, using 'sender' value 'Choice' & store the rows in df_choices
    df_info = df[df['sender'] == ("Form")]
    df_choices = df[df['sender'] == "Choice"]
    
    # Drop all irrelevant columns for both data bases 
    df_info = df_info.drop(['sender', 'sender_type', 'sender_id', 'timestamp', 'meta', 'duration', 'ended_on',
       'response', 'response_action', 'time_commit', 'time_end', 'time_render',
       'time_run', 'time_show', 'time_switch', 'url', 'Amount', 'Delay', 'ID'], axis = 1)
    
    df_choices = df_choices.drop(['sender', 'sender_type', 'sender_id', 'timestamp', 'meta', 'Age', 'Degree', 'Essentials', 'Happiness', 'ID',
       'Investments', 'Leisure', 'Sleep', 'Stress', 'response_action',  'time_commit', 'time_end', 'time_render', 'time_run', 'time_show',
       'time_switch', 'url'], axis = 1)
    
    # For df_info, we need to add in the RefID for future extraction
    # The file_count is used as a unique identifier here
    df_info['RefID'] = file_count
    
    # The base amount in all cases is $20, with 0 delay
    # This is added in to all rows, along with the unique identifier of the participant (RefID = file_count)
    df_choices['BaseAmount'] = 20.0
    df_choices['BaseDelay'] = 0
    df_choices['RefID'] = file_count
    
    # Convert all Delays to be in weeks
    dict = {"1 week" : '1', "2 weeks" : '2', "1 month": '4',"3 months": '12', "6 months" : '24', "1 year": '52'}
    df_choices=df_choices.replace({"Delay": dict})
    
    #Convert the response_time from milliseconds to seconds
    df_choices['duration_in_seconds'] = df_choices['duration'] / 1000.0
    
# Each participant has entered Financial Distribution data in a different way
# May be easiest to manually convert? 
    #     # Convert Financial Distribution data from % to float
    
#     df_info['Essentials'] = df_info['Essentials'].str.rstrip('%').astype('float') / 100.0
#     df_info['Investments'] = df_info['Investments'].str.rstrip('%').astype('float') / 100.0
#     df_info['Leisure'] = df_info['Leisure'].str.rstrip('%').astype('float') / 100.0
    
#     # Ensure that the distribution of expenses adds up to 1
#     df_info['SumOfExpenses'] = df_info['Essentials'] + df_info['Investments'] + df_info['Leisure']
#     df_info['valid'] = df_info['SumOfExpenses'] == 1.0
#     incorrect = df_info[df_info['valid'] == False]
    
#     # If the expenses don't add up, send a message to the console
#     if(not(incorrect.empty)):
#         print("Expenses don't add up!: \n", incorrect)
    
#     # Drop the additional columns after checking
#     df_info = df_info.drop(['SumOfExpenses', 'valid'], axis = 1)
    
    # If the page timed out i.e. the choice ended without the response being entered, send a message
    # to the console
    timed_out = df_choices[df_choices['ended_on'] != 'response']
    
    if(not(timed_out.empty)):
        print("Some responses were not collected because of a time out!: \n", timed_out)
    
    # Drop rows where choices timed out
    df_choices = df_choices[df_choices['ended_on'] == 'response']
    # After this, the 'ended_on' column does not have any further purpose
    df_choices = df_choices.drop(['ended_on'], axis = 1)


    
    # print(df_info.head())
    # print(df_choices.head())
    
    # Write off info & choice data
    df_info.to_csv("./../Cleaned_Data/Info{}.csv".format(file_count), index = False)
    df_choices.to_csv("./../Cleaned_Data/Choice_{}.csv".format(file_count), index = False)
        
    
    # For analysis, it may be easier to have all choice and participant information in a single dataframe
    # This is done after writing the choice data off to separate .csv
    df_choice_and_info = df_choices
    
    df_choice_and_info['Age'] = df_info['Age'].item()
    df_choice_and_info['Degree'] = df_info['Degree'].item()
    df_choice_and_info['Essentials'] = df_info['Essentials'].item()
    df_choice_and_info['Investments'] = df_info['Investments'].item()
    df_choice_and_info['Leisure'] = df_info['Leisure'].item()
    df_choice_and_info['Happiness'] = df_info['Happiness'].item()
    df_choice_and_info['Sleep'] = df_info['Sleep'].item()
    df_choice_and_info['Stress'] = df_info['Stress'].item()
    
    df_choice_and_info.to_csv("./../Cleaned_Data/Choice_And_Info_{}.csv".format(file_count), index = False)
    
    # Store all participant info, choice info, and combined info in separate databases
    # This will provide flexibility while performing data analysis
    # Code can be modified later according to the chosen Data Analysis Strategy 
    if(file_count == 1):
        df_all_info = df_info
        df_all_choice = df_choices
        df_all_choice_and_info = df_choice_and_info
    else:
        df_all_info = df_all_info.append(df_info)
        df_all_choice = df_all_choice.append(df_choices)
        df_all_choice_and_info = df_all_choice_and_info.append(df_choice_and_info)
        
    # Onto the next file!
    file_count = file_count + 1

In [93]:
# Write off all patient information to a separate .csv
df_all_info.to_csv('./../Combined_Data/ParticipantInfo.csv', index = False)
df_all_choice.to_csv('./../Combined_Data/ParticipantChoices.csv', index = False)
df_all_choice_and_info.to_csv('./../Combined_Data/ParticipantInfoAndChoices.csv', index = False)